In [ ]:
!pip install -q geopandas;
!pip install -q geemap;
!pip install -q --upgrade folium;
# !pip install sentinelhub;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... don

In [ ]:
import json
import ee, datetime, folium
import pandas as pd
from IPython.display import Image
from pylab import *
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from folium.plugins import HeatMap
import io
import fiona
import branca.colormap as cm
import geemap
# import geemap.foliumap
# import geemap.eefolium as geemap
from shapely.geometry import shape
from shapely.geometry import shape, Polygon, MultiPolygon

# Extract Satellite Indices per Geographic Area in England

## First authenticate GEE

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=js90zhZhrPJ4hEDbOH-B_mFpnLcAQvQq6RVqU2W9XzM&tc=xPhdi2tYMTRAoFY7ff_LSyG6pBj0wt26YlcrPPHiMXE&cc=dQZM9ucoRVXqy9p6wUHsHRgUGRHvRaIYazaMhM6cdBo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk6vSDWEF-XnL0PRcaqJ3kfHfRa8Axcl-edempztC1cdtSu_aEuf2uw

Successfully saved authorization token.


## Then also connect to Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Global Variables

In [ ]:
project_folder = "drive/MyDrive/PROJEKTI7S/AI4EO_Fellowship/Health/Satellite Health Correlates UK/"

!ls "drive/MyDrive/PROJEKTI7S/AI4EO_Fellowship/Health/Satellite Health Correlates UK/"

2.0_extract_AirQ_scores.ipynb  5.3_RF_data_log_normalized_standardized.ipynb
2.0_extract_NDVI_scores.ipynb  data
2.1_env_data_master.ipynb      materials
4.0_data_explore.ipynb	       old
5.1_LR_models.ipynb	       presentations
5.2_RF_no_data_scaling.ipynb   results


In [ ]:
# Define the coordinates for the city
city_name = 'London'
city_coordinates = (-0.1278, 51.5074)  # (lon, lat)

## 0 Functions

In [ ]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Define a function for displaying shapefiles saved into a feature collection
def plot_ee_feature_shapefile(ee_feature, zoom_start=7):
  # Get the bounds of the feature collection
  aoi = ee_feature.geometry()

  # Create a folium map object.
  center = aoi.centroid(10).coordinates().reverse().getInfo()
  m = folium.Map(location=center, zoom_start=zoom_start)

  # Add layers to the folium map.
  # Add the feature collection to the map as a GeoJSON layer
  m.add_child(folium.GeoJson(data=ee_feature.getInfo()))

  # Add a layer control panel to the map.
  m.add_child(folium.LayerControl())

  # Display the map.
  display(m)

In [ ]:
def plot_ee_feature_shapefile_geemap(fc, zoom_start=6, coordinates=(-0.1278, 51.5074), color=None):

  # Create a Map object
  Map = geemap.Map(center=[0, 0], zoom=zoom_start) # set initial center and zoom level

  # Add the Feature Collection to the Map, centered on the shapefile
  Map.addLayer(fc, {}, 'Shapefile')

  # too many edges say GEE
  # center = fc.geometry().centroid(100).coordinates().reverse().getInfo()
  # print (center)

  # # Automatically adjust the zoom level to fit the shapefile
  # Map.setCenter(lon=coordinates[0], lat=coordinates[1]) # center on the centroid

  Map.centerObject(fc, zoom_start)

  if color:
    # Add a color based on the 'means' property
    colorized = fc.map(lambda feature: feature.set({'style': {'color': 'blue', 'fillColor': feature.get(color)}}))

  # Add the colorized feature collection to the map
  Map.addLayer(colorized, {}, 'Colorized')

  display(Map)

In [ ]:
def spatial_join_ee_fc(ee_fc_fine, ee_fc_coarse):

  # Define the spatial join relationship
  spatialFilter = ee.Filter.intersects(
    leftField='.geo', # geometry field in regions feature collection
    rightField='.geo', # geometry field in wards feature collection
    maxError=10) # tolerance distance in meters

  # Perform the spatial join using the defined spatial filter
  ee_fc_joined = ee.Join.simple().apply(
    primary=ee_fc_fine,
    secondary=ee_fc_coarse,
    condition=spatialFilter)
  
  return ee_fc_joined

In [ ]:
def get_satellite_imagery(ee_the_region, sat, START_DATE, END_DATE):

  # these are parameters for selecting the Sentinel Image
  aoi = ee_the_region.geometry()

  # START_DATE = '2017-01-01' # inclusive
  # END_DATE = '2018-04-01' # exclusive
  CLOUD_FILTER = 1 # maximum amount of clouds in percent

  # Import and filter S2 collection
  s2_col = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(aoi)
      .filterDate(START_DATE, END_DATE)
      .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
  
  return s2_col

In [ ]:
def calculate_ndvi(image):
  return image.normalizedDifference(["B8", "B4"]).rename('NDVI')

In [ ]:
def efficent_fc_to_gdf(fc, selectors, batch_size=2000):

  print ("running efficient conversion to gdf")

  # Get the total number of features in the feature collection
  num_features = fc.size().getInfo()

  # Define a list to store the GeoDataFrames
  gdfs = []

  # Loop over the feature collection in batches of size batch_size
  for i in range(0, num_features, batch_size):
      print ("running efficient conversion to gdf. data {} of {}...".\
              format(i, int(num_features)))
      # Get the next batch of features using the limit() method
      # features = fc.limit(batch_size, str(i))
      features_list = fc.toList(batch_size, i)
      features = ee.FeatureCollection(features_list)

      # Convert the batch of features to a GeoDataFrame
      gdf = geemap.ee_to_geopandas(features, selectors=selectors)

      # Add the GeoDataFrame to the list
      gdfs.append(gdf)

  # Concatenate the GeoDataFrames into a single GeoDataFrame
  final_gdf = pd.concat(gdfs)

  # Display the GeoDataFrame
  # print(final_gdf.head())

  return final_gdf

In [ ]:
def extract_NVDI_ward_scores_region(region_name, ee_regions, ee_wards_UK, sat, START_DATE, END_DATE):

  year = START_DATE.split("-")[0]
  print("****** Processing {} in {} year *******".format(region_name, year))

  # prepare region geographies
  ee_the_region = ee_regions.filter(ee.Filter.eq('RGN22NM',region_name)) 
  ee_region_wards = spatial_join_ee_fc(ee_wards_UK, ee_the_region)
  print ("prepared Region geographies ...")

  # pull Sentinel-2 imagery
  s2_col = get_satellite_imagery(ee_the_region, sat, START_DATE, END_DATE)
  print ("pulled Sentinel-2 imagery ...")

  # calculate NVDI
  S2NDVI_col = s2_col.map(calculate_ndvi)
  S2NDVI = S2NDVI_col.median()
  print ("calculated Region NDVI ...")

  # take the Sentinel-2 NDVI we calculated
  ndvi = S2NDVI
  # get the shapefile we need
  shapefile = ee_region_wards
  # Get the mean value of each band within the shapefile
  means = ndvi.reduceRegions(collection=shapefile, reducer=ee.Reducer.mean(), scale=10)
  print ("calculated wards NDVI ...")

  selectors = ['wd19nm', 'wd19cd', 'mean']
  # Convert the feature collection to a GeoPandas dataframe
  gdf_means = geemap.ee_to_geopandas(means, selectors=selectors)
  # # Define the CRS of the GeoJSON data
  # gdf_means.crs = 'EPSG:4326'
  print ("converted feature collection to geopandas df ...")

  # save csv with results
  csv_results_address = project_folder + \
         "results/{}/ward/{}/{}.csv".format(sat, year, region_name)
  # create a Pandas dataframe with selected columns
  df_means = pd.DataFrame(gdf_means[selectors])	
  df_means.to_csv(csv_results_address, index=False) 
  print ("saved csv ...")

  return gdf_means

In [ ]:
def extract_NVDI_lsoa_scores_region(region_name, ee_regions, ee_lsoas_UK, sat, START_DATE, END_DATE):

  year = START_DATE.split("-")[0]
  print("****** Processing {} in {} year *******".format(region_name, year))

  # prepare region geographies
  ee_the_region = ee_regions.filter(ee.Filter.eq('RGN22NM',region_name)) 
  ee_region_lsoas = spatial_join_ee_fc(ee_lsoas_UK, ee_the_region)
  print ("prepared Region geographies ...")

  # pull Sentinel-2 imagery
  s2_col = get_satellite_imagery(ee_the_region, sat, START_DATE, END_DATE)
  print ("pulled Sentinel-2 imagery ...")

  # calculate NVDI
  S2NDVI_col = s2_col.map(calculate_ndvi)
  S2NDVI = S2NDVI_col.median()
  print ("calculated Region NDVI ...")

  # take the Sentinel-2 NDVI we calculated
  ndvi = S2NDVI
  # get the shapefile we need
  shapefile = ee_region_lsoas
  # Get the mean value of each band within the shapefile
  means = ndvi.reduceRegions(collection=shapefile, reducer=ee.Reducer.mean(), scale=50)
  print ("calculated LSOAs NDVI ...")

  selectors = ['LSOA21NM', 'LSOA21CD', 'mean']
  # Convert the feature collection to a GeoPandas dataframe
  gdf_means = efficent_fc_to_gdf(means, selectors=selectors)
  # gdf_means = geemap.ee_to_geopandas(means, selectors=selectors)
  # # Define the CRS of the GeoJSON data
  # gdf_means.crs = 'EPSG:4326'
  print ("converted feature collection to geopandas df ...")

  # save csv with results
  csv_results_address = project_folder + \
         "results/{}/lsoa/{}/{}.csv".format(sat, year, region_name)
  # create a Pandas dataframe with selected columns
  df_means = pd.DataFrame(gdf_means[selectors])		
  df_means.to_csv(csv_results_address, index=False) 
  print ("saved csv ...")

  return gdf_means

## 1 Download / Read In Data

## Spatial Data

### Region Data

In [ ]:
# England regions data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/ons::regions-december-2022-en-bfc/explore?location=52.916331%2C-2.000000%2C7.74

drive_address = project_folder + "data/England-regions/Regions_(December_2022)_EN_BFC.shp"
asset_address = "projects/sanja7s-ee/assets/England_regions"


# 1 we can read them into geopandas from Drive
regions = gpd.read_file(drive_address)
print("There are {} regions in England.".format(len(regions)))

# 2 we can read the data as a feature shapefile from GEE assets
ee_regions = ee.FeatureCollection(asset_address)

print (regions.crs)
# regions.head()

There are 9 regions in England.
EPSG:27700


### Ward Data

In [ ]:
# UK wards data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/search?collection=Dataset&sort=name&tags=all(BDY_WD)

drive_address = project_folder + "data/UK-wards/WD_DEC_2019_GB_BFE/geodb_DBO_WD_DEC_2019_GB_BFE.shp"
asset_address = "projects/sanja7s-ee/assets/UK_wards"


# # 1 we can read them into geopandas from Drive
wards_UK = gpd.read_file(drive_address)
wards = wards_UK[wards_UK['wd19cd'].str.startswith('E')]
print("There are {} wards in England out of {} in the UK.".format(len(wards), len(wards_UK)))

# # 2 we can read the data as a feature shapefile from GEE assets
ee_wards_UK = ee.FeatureCollection(asset_address)

# # 3 join wards with regions
wards_regions = gpd.sjoin(wards, regions, predicate='within')
ee_wards_regions = spatial_join_ee_fc(ee_wards_UK, ee_regions)

# print (wards.crs)
# wards.head()

There are 30 wards in England out of 30 in the UK.


<ipython-input-21-4c1c2e652303>:17: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:27700

  wards_regions = gpd.sjoin(wards, regions, predicate='within')


In [ ]:
# plot_ee_feature_shapefile_geemap(ee_wards_regions, zoom_start=7)

### LSOAs Data

In [ ]:
# UK LSOAs data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/766da1380a3544c5a7ca9131dfd4acb6_0/explore?location=51.420294%2C-0.114413%2C11.00

drive_address = project_folder + "data/UK-LSOAs/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC).shp"
asset_address = "projects/sanja7s-ee/assets/UK_LSOAs"


# 1 we can read them into geopandas from Drive
lsoas_UK = gpd.read_file(drive_address)
lsoas = lsoas_UK[lsoas_UK['LSOA21CD'].str.startswith('E')]
print("There are {} wards in England out of {} in the UK.".format(len(lsoas), len(lsoas_UK)))

# 2 we can read the data as a feature shapefile from GEE assets
ee_lsoas_UK = ee.FeatureCollection(asset_address)
# # 3 join wards with regions
lsoas_regions = gpd.sjoin(lsoas, regions, predicate='within')
ee_lsoas_regions = spatial_join_ee_fc(ee_lsoas_UK, ee_regions)

# lsoas.head()
print (lsoas.crs)

There are 33755 wards in England out of 35672 in the UK.
EPSG:27700


In [ ]:
# plot_ee_feature_shapefile_geemap(ee_wards_regions, zoom_start=7)

# Process a Region 

In [ ]:
# ward_NDVI_scores = extract_NVDI_ward_scores_region(region_name='West Midlands',\
#                                ee_regions=ee_regions, ee_wards_UK=ee_wards_UK,\
#                             sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

# Process all regions

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#   ward_NDVI_scores = extract_NVDI_ward_scores_region(region_name=region_name, ee_regions=ee_regions, ee_wards_UK=ee_wards_UK)

In [ ]:
for region_name in regions['RGN22NM'].unique():
  ward_NDVI_scores = extract_NVDI_ward_scores_region(region_name=region_name,\
                              ee_regions=ee_regions, ee_wards_UK=ee_wards_UK,\
                            sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

# Process an LSOA 

In [ ]:
lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name='London',\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing London *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 5128...
running efficient conversion to gdf. data 2000 of 5128...
running efficient conversion to gdf. data 4000 of 5128...
converted feature collection to geopandas df ...
saved csv ...


In [ ]:
lsoa_NDVI_scores.drop_duplicates()

,geometry,GlobalID,LSOA21CD,LSOA21NM,OBJECTID,Shape__Are,Shape__Len,mean
0,"POLYGON ((-0.33423 51.35454, -0.33388 51.35357...",32b563ec-d48e-47f9-a28f-d50a9d2c070f,E01030296,Elmbridge 015A,28745,1.661265e+06,5886.233833,0.692033
1,"POLYGON ((-0.31633 51.37001, -0.31593 51.37213...",ef196942-c797-46be-9eba-9726dd761338,E01030319,Elmbridge 009A,28768,1.471392e+06,7315.771514,0.645321
2,"POLYGON ((-0.32441 51.39161, -0.32372 51.38784...",0e307556-7803-445c-9711-f80562647d61,E01030322,Elmbridge 006A,28771,3.748679e+05,2580.515371,0.434842
3,"POLYGON ((-0.32024 51.38687, -0.32021 51.38599...",d71c8af2-4a50-44d7-9f55-3af9dc3c9918,E01030323,Elmbridge 006B,28772,4.234492e+05,4197.344424,0.646703
4,"POLYGON ((-0.31110 51.37492, -0.30910 51.37526...",b21ac6ce-f027-456e-8b65-6656cb66429b,E01030325,Elmbridge 006D,28774,1.047266e+06,5859.883653,0.650938
...,...,...,...,...,...,...,...,...
1123,"POLYGON ((-0.19834 51.51702, -0.19585 51.51737...",fd1e73e6-bae0-49e0-be54-4aafc5f2d7b2,E01004760,Westminster 014E,4502,7.073876e+04,1404.749484,0.348377
1124,"POLYGON ((-0.15002 51.50204, -0.14830 51.50271...",3904c1a3-077d-4dfe-b228-11a1e09a34a4,E01004761,Westminster 018D,4503,5.192516e+05,5584.410109,0.140346
1125,"POLYGON ((-0.15678 51.51359, -0.15577 51.51168...",13c7b484-68b0-4100-bd27-cb5069c6bbfe,E01004762,Westminster 011E,4504,2.981196e+05,4146.018005,0.113364
1126,"POLYGON ((-0.13452 51.51913, -0.13497 51.51923...",cd0dc731-4ee0-42a1-8686-0f7149a3a095,E01004763,Westminster 013B,4505,2.939641e+05,4374.422398,0.119234


# Process all LSOAs

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#     lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK)

2021 - 2022

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name,\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing North East in 2021 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2021 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2021 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to

2017 - 2018 (the earliest possible with this satellite)

In [ ]:
START_DATE = '2017-01-01' # inclusive
END_DATE = '2018-04-01' # exclusive

for region_name in regions['RGN22NM'].unique():
    lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name,\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='S2', START_DATE=START_DATE, END_DATE=END_DATE)

****** Processing North East in 2017 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2017 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2017 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to